In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

# turns off warnings
import warnings
warnings.filterwarnings('ignore')


def check_acc(a, b, test, correct):
    for i in range(len(test)):
        row = test.iloc[i]
        compare = list(correct)[i]
        correct_ctr = 0
        
        result = np.dot(row, a) + b
        if((result < 0 and compare < 0) or (result > 0 and compare > 0)):
            correct_ctr += 1
        
    return correct_ctr / len(test)


def calc_gradient(a, b, x, y_k, reg):
    check = np.dot(a, x) + b
    if(y_k * check >= 1):
        return (reg*a, 0)
    else:
        return ((reg*a - (y_k*x)), -1 * y_k)
    

In [2]:
df = pd.read_csv('20K_K9.csv', dtype=np.float64, header=None)
df = df.drop(df.columns[0], axis=1)
df.columns = np.arange(0, 5409)
df

0      1      2      3      4      5      6      7      8      9     \
0     -0.161 -0.014  0.002 -0.036 -0.033 -0.093  0.025  0.005  0.000 -0.015   
1     -0.158 -0.002 -0.012 -0.025 -0.012 -0.106  0.013  0.005  0.000 -0.002   
2     -0.169 -0.025 -0.010 -0.041 -0.045 -0.069  0.038  0.014  0.008 -0.014   
3     -0.183 -0.051 -0.023 -0.077 -0.092 -0.015  0.071  0.027  0.020 -0.019   
4     -0.154  0.005 -0.011 -0.013 -0.002 -0.115  0.005  0.002 -0.003  0.002   
5     -0.150  0.016 -0.014  0.000  0.016 -0.123 -0.004 -0.002 -0.005  0.010   
6     -0.158  0.002 -0.019 -0.028 -0.008 -0.101  0.011  0.005  0.001  0.003   
7     -0.152  0.009 -0.015 -0.008  0.004 -0.120 -0.002 -0.002 -0.007  0.004   
8     -0.172 -0.028  0.003 -0.045 -0.055 -0.078  0.039  0.011  0.006 -0.020   
9     -0.164 -0.019 -0.011 -0.037 -0.031 -0.081  0.029  0.012  0.007 -0.010   
10    -0.148  0.018 -0.012  0.004  0.021 -0.128 -0.007 -0.003 -0.006  0.010   
11    -0.153  0.012 -0.017 -0.009  0.006 -0.116  0.000  0.001 -0.004  0.008   
12    -0.161 -0.014  0.001 -0.033 -0.027 -0.093  0.023  0.007  0.003 -0.013   
13    -0.153  0.010 -0.017 -0.012  0.004 -0.112  0.004  0.002 -0.002  0.007   
14    -0.153  0.006 -0.011 -0.010  0.000 -0.119  0.003  0.000 -0.005  0.004   
15    -3.272 -0.168  0.113  0.064  0.057 -0.050  0.084  0.097  0.089 -0.078   
16    -0.141  0.035 -0.015  0.023  0.059 -0.143 -0.018 -0.006 -0.010  0.018   
17    -0.166 -0.019  0.000 -0.041 -0.038 -0.093  0.029  0.008  0.003 -0.017   
18    -0.175 -0.036 -0.022 -0.058 -0.061 -0.043  0.052  0.022  0.015 -0.014   
19    -0.158 -0.004 -0.014 -0.026 -0.011 -0.099  0.013  0.006  0.002  0.000   
20    -0.140  0.043 -0.015  0.033  0.080 -0.148 -0.027 -0.011 -0.014  0.023   
21    -0.164 -0.013 -0.005 -0.032 -0.028 -0.100  0.023  0.007  0.002 -0.012   
22    -0.142  0.039 -0.016  0.028  0.071 -0.144 -0.022 -0.009 -0.012  0.021   
23    -0.156 -0.002 -0.010 -0.018 -0.009 -0.108  0.011  0.003 -0.002 -0.002   
24    -0.148  0.024 -0.015  0.008  0.033 -0.129 -0.010 -0.003 -0.008  0.013   
25    -0.160 -0.005 -0.013 -0.031 -0.016 -0.096  0.017  0.007  0.002 -0.003   
26    -0.146  0.026 -0.015  0.010  0.037 -0.131 -0.011 -0.003 -0.008  0.014   
27    -0.168 -0.019 -0.010 -0.037 -0.039 -0.074  0.033  0.012  0.007 -0.013   
28    -0.155  0.002 -0.010 -0.016 -0.009 -0.111  0.009  0.003 -0.001 -0.001   
29    -0.169 -0.022  0.002 -0.040 -0.041 -0.095  0.031  0.008  0.003 -0.019   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19773  0.002  0.009  0.001  0.015  0.025 -0.028 -0.009 -0.005 -0.004  0.003   
19774 -0.004 -0.020  0.014 -0.016 -0.038  0.040  0.017  0.003  0.004 -0.011   
19775  0.003  0.026 -0.004  0.032  0.064 -0.058 -0.023 -0.011 -0.010  0.010   
19776  0.000  0.001  0.005  0.004  0.000 -0.004 -0.001 -0.002 -0.001 -0.001   
19777  0.000  0.001  0.003  0.006  0.008 -0.035 -0.001 -0.002 -0.002  0.000   
19778  0.001  0.004  0.001  0.007  0.011 -0.013 -0.005 -0.003 -0.002  0.000   
19779  0.000  0.001  0.005  0.008  0.006 -0.005 -0.002 -0.002 -0.002  0.000   
19780 -0.015  0.012  0.046 -0.034 -0.105  0.132  0.055  0.011  0.011 -0.035   
19781 -0.002 -0.013  0.010 -0.014 -0.030  0.031  0.012  0.002  0.004 -0.008   
19782  0.001 -0.002  0.004  0.002 -0.002  0.001  0.001 -0.001  0.001 -0.001   
19783 -0.006 -0.026 -0.001 -0.017 -0.031  0.015  0.015  0.007  0.005 -0.010   
19784 -0.011 -0.038 -0.007 -0.030 -0.041  0.025  0.022  0.009  0.007 -0.014   
19785 -0.015 -0.012 -0.054 -0.021  0.038 -0.057  0.002  0.008  0.002  0.002   
19786 -0.008 -0.037 -0.010 -0.031 -0.037 -0.012  0.017  0.007  0.007 -0.012   
19787 -0.001  0.016 -0.015  0.014  0.048 -0.046 -0.014 -0.005 -0.005  0.008   
19788 -0.011 -0.046 -0.009 -0.052 -0.053  0.073  0.036  0.014  0.013 -0.016   
19789 -0.260 -0.138 -0.167 -0.160 -0.078 -0.025  0.075  0.061  0.046 -0.026   
19790 -0.001 -0.008 -0.009 -0.005  0.004 -0.255  0.003  0.002  0.002  0.000   
19791 -0.009 -0.055  0.012 -0.054 -0.064  0.081  0.032  0.

In [3]:
# drop last colmun and replace active/inactive with numbers
X = df.drop(df.columns[-1], axis=1, inplace=False)
y = df.iloc[:,-1]

In [4]:
# split into training and eval
X_tru, X_gar, y_tru, y_gar = train_test_split(X, y, train_size=.1)
X_t, X_eval, y_t, y_eval = train_test_split(X_tru, y_tru, test_size=0.1)
X_t

0      1      2      3      4       5       6      7      8      9     \
12440 -0.162 -0.017  0.006 -0.036 -0.044  -0.230   0.034  0.010  0.005 -0.015   
2362  -0.160 -0.005 -0.008 -0.024 -0.018  -0.108   0.015  0.004 -0.001 -0.006   
16748 -0.002 -0.021  0.012 -0.011 -0.032   0.037   0.011  0.002  0.003 -0.012   
15140  0.856  2.722  1.854  1.147  0.657  19.053  19.967  1.553  2.116  2.630   
17264 -0.002 -0.007  0.008 -0.002 -0.017   0.009   0.005  0.000  0.001 -0.005   
4522  -0.146  0.027 -0.014  0.013  0.039  -0.131  -0.013 -0.005 -0.009  0.014   
3902  -0.167 -0.019 -0.003 -0.037 -0.037  -0.090   0.031  0.009  0.004 -0.016   
14754 -0.171 -0.028  0.000 -0.045 -0.053  -0.077   0.038  0.012  0.007 -0.020   
855   -0.154  0.006 -0.012 -0.011 -0.001  -0.117   0.004  0.002 -0.003  0.002   
8255  -0.147  0.025 -0.014  0.010  0.036  -0.132  -0.011 -0.004 -0.008  0.014   
3582  -0.184 -0.055  0.004 -0.082 -0.110   0.001   0.074  0.021  0.016 -0.029   
2534  -0.168 -0.025 -0.011 -0.041 -0.043  -0.071   0.037  0.014  0.009 -0.013   
4327  -0.164 -0.021 -0.001 -0.038 -0.036  -0.089   0.031  0.009  0.004 -0.016   
16776  0.000 -0.001  0.007  0.004 -0.002  -0.010   0.000 -0.001 -0.001 -0.003   
5701  -0.153  0.008 -0.012 -0.010  0.001  -0.115   0.004  0.002 -0.003  0.003   
1978  -0.165 -0.016 -0.012 -0.037 -0.032  -0.084   0.028  0.012  0.006 -0.010   
3     -0.183 -0.051 -0.023 -0.077 -0.092  -0.015   0.071  0.027  0.020 -0.019   
12117 -0.168 -0.027 -0.011 -0.042 -0.045  -0.067   0.039  0.015  0.009 -0.014   
16248 -0.163 -0.016 -0.010 -0.037 -0.030  -0.083   0.028  0.010  0.005 -0.013   
4093  -0.162 -0.012 -0.019 -0.038 -0.020  -0.086   0.024  0.012  0.005 -0.004   
15354 -0.157  0.000 -0.017 -0.027 -0.009  -0.099   0.013  0.006  0.001  0.000   
4132  -0.150  0.011 -0.008 -0.011 -0.001  -0.133   0.011  0.005  0.001 -0.007   
4741  -0.176 -0.035 -0.014 -0.067 -0.078  -0.049   0.070  0.025  0.022 -0.026   
8917  -0.157 -0.001 -0.009 -0.018 -0.010  -0.110   0.011  0.003 -0.003 -0.002   
14694 -0.148  0.021 -0.015  0.006  0.030  -0.128  -0.008 -0.003 -0.008  0.012   
4962  -0.159 -0.008 -0.010 -0.027 -0.016  -0.099   0.018  0.006  0.001 -0.005   
5782  -0.166 -0.022 -0.015  0.204  0.336  -0.071   0.034  0.015  0.010 -0.007   
3970  -0.163 -0.015 -0.009  0.216  0.347  -0.083   0.027  0.012  0.007 -0.009   
8890   0.102  0.161  0.123  0.037 -0.050   0.110   0.054  0.017  0.014 -0.021   
8273  -0.165 -0.017 -0.012 -0.038 -0.031  -0.083   0.029  0.011  0.005 -0.010   
...      ...    ...    ...    ...    ...     ...     ...    ...    ...    ...   
5876  -0.162 -0.009 -0.024 -0.040 -0.015  -0.092   0.014  0.009  0.004 -0.003   
16024 -0.178 -0.036  0.022 -0.052 -0.072  -0.093   0.045  0.005  0.000 -0.032   
19152 -0.006 -0.042  0.001 -0.057 -0.064   0.096   0.037  0.011  0.012 -0.017   
18997  0.003  0.037  0.000  0.029  0.056  -0.074  -0.016 -0.009 -0.007  0.002   
9458  -0.166 -0.017 -0.010 -0.038 -0.034  -0.047   0.029  0.011  0.006 -0.012   
13428 -0.159 -0.007 -0.008 -0.026 -0.018  -0.104   0.016  0.006  0.000 -0.006   
7808  -0.172 -0.032 -0.018 -0.050 -0.052  -0.052   0.047  0.020  0.014 -0.013   
1929  -0.179 -0.031  0.001 -0.051 -0.062  -0.097   0.043  0.011  0.005 -0.025   
1464  -0.145  0.023 -0.005 -0.043 -0.145  -0.136  -0.010  0.000 -0.004  0.013   
18168 -0.007 -0.047  0.014 -0.060 -0.076   0.133   0.047  0.016  0.016 -0.019   
13270 -0.151  0.012 -0.014 -0.006  0.007  -0.118   0.000  0.000 -0.004  0.007   
16986 -0.009 -0.039  0.038 -0.048 -0.075   0.080   0.034  0.004  0.007 -0.028   
15110 -0.149  0.019 -0.017  0.001  0.021  -0.125  -0.006 -0.002 -0.005  0.011   
579   -0.165 -0.020  0.002 -0.037 -0.039  -0.089   0.029  0.007  0.003 -0.017   
11195 -0.172 -0.030 -0.014 -0.048 -0.053  -0.060   0.044  0.017  0.011 -0.015   
9532  -0.178 -0.030 -0.004 -0.055 -0.060  -0.102   0.042  0.015  0.011 -0.021   
16732 -0.001 -0.006  0.009 -0.001 -0.015   0.006   0.004 -0.001  0.000 -0.004   
9840  -0.161 -0.010 -0.005 -0.030 -0.02

In [7]:
# split remaining into training and test data
reg_val = np.array([.0001, .001, .01, .1])

#Get a random val
random_value= int(np.random.random()*X_t[1].count())

#Generate initial values for a & b.
a_vec = X_t.iloc[random_value]
b = list(y_t)[random_value]


pk = np.full((5408,), 1)
m = 10
n = 1
tolerance = 1e-6

# (a, b, accuracy, lambda)
best = tuple()
best_list = list()

X_training, X_test, y_training, y_test = train_test_split(X_t, y_t, test_size=0.4)
#     print(type(X_test.ix[1]))
#     break

# normalize the data set X_training
cols = list(X_training.columns)
for col in cols:
    X_training[col] = (X_training[col] - X_training[col].mean()) / X_training[col].std(ddof=1)

# number of (rows * 2) / 100
#steps = (X_training[0].count() * 2) / 100
steps = 200

In [8]:
for lam in reg_val:
    #100 seasons for each lambda.
    for i in range(100):
        
        for j in range(int(steps)):
            # creates some random row from our training set. (sampling with replacement.)
            rand_val = int(np.random.random()*X_training[1].count())
            rand_row = X_training.iloc[rand_val]
        
            #target_val = y_training[rand_val]
            #Get the one value
            target_val = list(y_training)[rand_val]

            
            # grad is a tuple of the adjusted value for a and b
            grad = calc_gradient(a_vec, b, rand_row, target_val, lam)
            step_size = (m / (j + n))
            
            #Update the gradient values.
            a_vec = a_vec - (step_size * grad[0])
            b = b - (step_size * grad[1])
            accuracy = check_acc(a_vec, b, X_test, y_test)

            #Check the curr value.
            curr = tuple([a_vec, b, accuracy, lam])
            best_list.append(curr)
            if(len(best) == 0 or best[2] < accuracy):
                best = curr
                print(curr[2])
                
    

0.0
0.001402524544179523


In [ ]:
#Declare an Naive Bayes implementation for comparison

gnb = GaussianNB()
#Use the ShuffleSplit cross validation module to 
cv = ShuffleSplit(n_splits=n_folds, test_size=0.1)
scores = cross_val_score(gnb, X, y, cv=cv)
    
#Output the means and std deviations.
print("The mean of the accuracy is: {0}".format(np.mean(scores)))
print("The Standard Deviation of the accuracy is: {0}".format(np.std(scores)))

In [10]:
for i in best_list:
    print(i[2])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
